## Урок 13. Модель BERT и GPT
## Практическое задание

Задание

1. Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

2. (дополнительно необязательная задача)на выбор взять
https://huggingface.co/datasets/sberquad
https://huggingface.co/datasets/blinoff/medical_qa_ru_data
натренировать любую модель для вопросно ответной системы
как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [1]:
!pip install transformers sentencepiece --quiet

     |████████████████████████████████| 4.7 MB 35.7 MB/s 
     |████████████████████████████████| 1.3 MB 47.7 MB/s 
     |████████████████████████████████| 120 kB 66.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.3 MB/s 


In [2]:
!pip install datasets --quiet

     |████████████████████████████████| 365 kB 34.5 MB/s 
     |████████████████████████████████| 212 kB 69.3 MB/s 
     |████████████████████████████████| 115 kB 49.3 MB/s 
     |████████████████████████████████| 127 kB 62.8 MB/s 


In [3]:
from datasets import load_dataset

corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-3b40850a8ccdf74d/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'paraphrases'],
        num_rows: 1725393
    })
})

In [5]:
corpus['train'][0:4]

{'id': ['0', '1', '2', '3'],
 'paraphrases': [['Птичий праздник: Что такое "куриное рождество" и кто его отмечает',
   'Куриное Рождество в 2019 году: что это за праздник и как стать его главным героем'],
  ['В Индии 5 млн женщин выстроились стеной в 620 км ради равенства (Фото)',
   'Индийские женщины выстроили живую стену длиной 620 километров, требуя равенства',
   'В Индии женщины встали живой стеной длиной 620 км'],
  ['Томас Бах: "Олимпийский комитет России понес достаточное наказание"',
   'Президент МОК Бах: Россия уже достаточно наказана за допинг',
   'Глава МОК заявил, что ОКР понес наказание за манипуляции с пробами на ОИ в Сочи'],
  ['Россия не представила WADA данные по пробам в срок и может быть наказана',
   'WADA: Россия не предоставила нам доступ к допинг-пробам',
   'В WADA пожаловались на отсутствие доступа к допинг-пробам российских спортсменов']]}

## Используем предобученную модель 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

In [6]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

In [7]:
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
        
    p = proba[0]  # p(non-paraphrase), p(paraphrase)
    if p[1] > 0.5:
        paraphrase = "paraphrase"
    else:
        paraphrase = "non"
        
    return [text1, text2, p[0], p[1], paraphrase]

In [8]:
print(compare_texts('Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода'))
# [0.7056226 0.2943774]
print(compare_texts('Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась'))

['Сегодня на улице хорошая погода', 'Сегодня на улице отвратительная погода', 0.7056226, 0.29437742, 'non']
['Сегодня на улице хорошая погода', 'Отличная погодка сегодня выдалась', 0.1652439, 0.83475614, 'paraphrase']


## Используем предобученную модель 'cointegrated/rut5-base-paraphraser'

In [10]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name_gen = 'cointegrated/rut5-base-paraphraser'
model_gen = T5ForConditionalGeneration.from_pretrained(model_name_gen)
tokenizer = T5Tokenizer.from_pretrained(model_name_gen)
model_gen.cuda();
model_gen.eval();

In [11]:
def get_paraphrase(text, beams=5, grams=4, do_sample=False):

    x = tokenizer(text, return_tensors='pt', padding=True).to(model_gen.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model_gen.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)

    return tokenizer.decode(out[0], skip_special_tokens=True)

In [13]:
print(get_paraphrase('Сегодня на улице хорошая погода.'))

Сегодня погода на улице очень хорошая.
